In [98]:
from python_graphql_client import GraphqlClient
import datetime

# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v3/graphql")

#timestamp = "2021-08-25T15:52:43.785+02:00"
timestamp = datetime.datetime.now().astimezone().isoformat()

# Create the query string and variables required for the request.
query = """
query t($fromPlace: String, $toPlace: String, $timestamp: DateTime){
  trip(
    from: {place: $fromPlace}
    to: {place: $toPlace}
    numTripPatterns: 3
    dateTime: $timestamp
    walkSpeed: 1.3
    arriveBy: false
  ) {
    tripPatterns {
      expectedStartTime
      legs {
        mode
        toPlace {
          name
        }
        interchangeTo {
          guaranteed
          ToServiceJourney {
            privateCode
          }
        }
        interchangeFrom {
          guaranteed
          ToServiceJourney {
            id
          }
        }
      }
    }
 }
}
"""


variables = {
    "fromPlace": "NSR:Quay:99503",
    "toPlace": "NSR:Quay:72050",
    "timestamp": timestamp,
}

data = client.execute(query=query, variables=variables)
data

{'data': {'trip': {'tripPatterns': [{'expectedStartTime': '2021-08-24T12:58:47+02:00',
     'legs': [{'mode': 'foot',
       'toPlace': {'name': 'Trondheim lufthavn'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Buran'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Siemens'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'foot',
       'toPlace': {'name': 'Siemens'},
       'interchangeTo': None,
       'interchangeFrom': None}]},
    {'expectedStartTime': '2021-08-24T13:18:47+02:00',
     'legs': [{'mode': 'foot',
       'toPlace': {'name': 'Trondheim lufthavn'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Buran'},
       'interchangeTo': None,
       'interchangeFrom': None},
      {'mode': 'bus',
       'toPlace': {'name': 'Siemens'},
       'interc

In [101]:
for i in data['data']:
    trippattern = data['data'][i]['tripPatterns']
    for pattern in trippattern:
        print(pattern['expectedStartTime'])
        legs = pattern['legs']
        for leg in legs:
            # {'mode': 'bus', 'toPlace': {'name': 'Dronningens gate'}, 'interchangeTo': {'guaranteed': True, 'ToServiceJourney': {'privateCode': '65'}}, 'interchangeFrom': None}

            #print(leg)
            t = f'\tleg {leg["mode"]} to "{leg["toPlace"]["name"]}"'
            
            interchangeTo = leg['interchangeTo']
            if interchangeTo and interchangeTo['guaranteed'] == True:
                t += f' guaranteed to {interchangeTo["ToServiceJourney"]["privateCode"]}'

            interchangeFrom = leg['interchangeFrom']
            if interchangeFrom and interchangeFrom['guaranteed'] == True:
                t += f' guaranteed from {interchangeFrom["ToServiceJourney"]["id"]}'

            print(t)


2021-08-24T12:58:47+02:00
	leg foot to "Trondheim lufthavn"
	leg bus to "Buran"
	leg bus to "Siemens"
	leg foot to "Siemens"
2021-08-24T13:18:47+02:00
	leg foot to "Trondheim lufthavn"
	leg bus to "Buran"
	leg bus to "Siemens"
	leg foot to "Siemens"
2021-08-24T13:21:10+02:00
	leg foot to "Sandfærhus"
	leg bus to "Siemens"
	leg foot to "Siemens"


In [97]:
query = """{
line(id: "ATB:Line:2_3") {
    id
    name
    quays {
      id
      name
    }
  }}
"""

data = client.execute(query=query, variables=variables)
data

{'data': {'line': {'id': 'ATB:Line:2_3',
   'name': 'Lohove- Sentrum- Hallset',
   'quays': [{'id': 'NSR:Quay:102729', 'name': 'Hallset'},
    {'id': 'NSR:Quay:102734', 'name': 'Arnt Smistads veg'},
    {'id': 'NSR:Quay:75528', 'name': 'Byåsen skole'},
    {'id': 'NSR:Quay:74073', 'name': 'Stabells veg'},
    {'id': 'NSR:Quay:72856', 'name': 'Havstadsenteret'},
    {'id': 'NSR:Quay:73843', 'name': 'Johan Falkbergets vei'},
    {'id': 'NSR:Quay:72800', 'name': 'Åsveien skole'},
    {'id': 'NSR:Quay:71634', 'name': 'Tvetestien'},
    {'id': 'NSR:Quay:74330', 'name': 'Nyveibakken'},
    {'id': 'NSR:Quay:72052', 'name': 'Ila'},
    {'id': 'NSR:Quay:72212', 'name': 'Skansen'},
    {'id': 'NSR:Quay:71685', 'name': 'Hospitalskirka'},
    {'id': 'NSR:Quay:74264', 'name': 'Kongens gate'},
    {'id': 'NSR:Quay:71174', 'name': 'Nidarosdomen'},
    {'id': 'NSR:Quay:73102', 'name': 'Studentersamfundet'},
    {'id': 'NSR:Quay:71940', 'name': 'Høgskoleringen'},
    {'id': 'NSR:Quay:75707', 'name': 'G

In [ ]:
lines = [1,1,10,2,3,310,311,340,350,410,420,421,422,430,440,451,452,455,460,470,480,490,5001,5005,5006,504,505,5100,5101,5102,5103,5104,5106,511,5110,5205,5206,5207,5209,5210,5305,535,5400,5402,5404,541,543,5471,5473,5475,5476,552,5521,5601,5602,5603,5604,561,5610,5701,5703,5704,5707,5801,5802,5805,5904,591,592,593,594,595,610,611,612,615,630,635,640,660,665,670,680,690,695,71,72,7201,7204,7205,7206,7209,7210,7211,722,723,7301,7302,7304,7306,7307,731,7311,7312,7313,7315,7317,7318,7323,7328,7329,733,7330,7402,7404,7408,741,7410,7412,7414,7415,7416,7419,7424,7425,75,7503,7504,7507,7510,7515,7519,7520,7521,76,7603,762,763,7702,7703,7705,7708,781,7811,782,783,784,785,7901,7902,7903,7904,7905,7908,80,83,14,41,44,46,70,73,74,77,78,79,40,42,7701,5472,5501,5211,5212,5300,5401,5406,5004,5111,544,5605,5702,7509,7814,82,5533,7208,7341,7308,7310,7322,7327,7405,7403,7411,7417,7420,7506,7710,7910,214]


query = """
{
  line(id: "ATB:Line:2_310943") {
    id
    name
  }
}
"""

data = client.execute(query=query, variables=variables)
data